# MEng Computer Science Thesis Implementation

**Title:** Benchmarking Persian Instruction-Tuned LLMs Versus State-of-the-Art NLP Models for Sentiment Analysis <br/>
**Student Name:** Seyyed Jalal Tabatabaee <br/>
**Student ID:** GH1033801 <br/>
**Supervisor:** Prof. Dr. Reza Babaei <br/>
**Semester:** Summer 2025 <br/>
**University:** Gisma University of Applied Sciences <br/>
**Github Link:** https://github.com/shayantabatabaee/llm-sentiment-analysis <br/>
**Dataset Link:** https://github.com/persiannlp/parsinlu



### Introduction

This project focuses on sentiment analysis for the Persian language, a low-resource language that has been underexplored in modern NLP research. While most sentiment analysis studies have been conducted in English, Persian poses unique challenges due to its complex grammar, diverse vocabulary, and the absence of diacritics.

Traditional approaches to sentiment analysis involve steps like text preprocessing, embedding generation, and classification using machine learning (ML) or deep learning (DL) models. However, with the emergence of large language models (LLMs) such as ChatGPT, new techniques like zero-shot and few-shot prompting have shown promising results — even without fine-tuning.

This research benchmarks instruction-tuned Persian LLMs (with fewer parameters) on the ParsiNLU sentiment dataset and compares their performance against state-of-the-art traditional models like BERT. The goal is to investigate whether smaller, Persian-specific LLMs can offer performance comparable to larger multilingual models while being more cost-efficient and practical for real-world applications.



### Install Libraries

In [ ]:
!pip install -q datasets
!pip install -q pandas

### Import Libraries

In [ ]:
from datasets import load_dataset
from IPython.display import display
import pandas as pd

### Initialize Variables

In [ ]:
DATASET_URL = "https://raw.githubusercontent.com/persiannlp/parsinlu/master/data/sentiment-analysis/food.jsonl"
LABEL_MAP = {
    "3": "OTHER",
    "2": "POSITIVE",
    "1": "POSITIVE",
    "0": "NEUTRAL",
    "-1": "NEGATIVE",
    "-2": "NEGATIVE"
}
TEST_COUNT = 190
RANDOM_STATE = 18

In [ ]:
label_en_fa = {
    "OTHER": "سایر",
    "POSITIVE": "مثبت",
    "NEUTRAL": "خنثی",
    "NEGATIVE": "منفی"
}

## Dataset Preprocessing

To ensure a direct comparison with the baseline study [(Abaskohi et al., 2024)](https://arxiv.org/pdf/2404.02403), we used the same dataset and fixed the random seed. Additionally, we merged the original six sentiment categories into four, matching the label structure used in the baseline. The original dataset contains six labels: *VERY_POSITIVE, POSITIVE, NEUTRAL, NEGATIVE, VERY_NEGATIVE,* and *OTHER*. For our analysis, we merged *VERY_POSITIVE* with *POSITIVE* and *VERY_NEGATIVE* with *NEGATIVE*.

### Load Dataset

In [ ]:
dataset = load_dataset("json", data_files=DATASET_URL)
df = dataset['train'].to_pandas()
df = df[['review', 'sentiment']]
df['sentiment'] = df['sentiment'].map(LABEL_MAP)

In [ ]:
sub_df = df.sample(n=TEST_COUNT, random_state=RANDOM_STATE)

### Visualize Dataset

In [ ]:
def insight(data):
  print(f"Dataset shape: {data.shape}", end="\n\n")
  print("Five sample of data")
  display(data.head(5))
  print("\n\nDataset info", end="\n\n")
  display(data['sentiment'].value_counts())
  data['sentiment'].value_counts().plot.bar(x="Sentiment", y="Count", rot=45)

In [ ]:
insight(df)

In [ ]:
insight(sub_df)

## Prompts

In [ ]:
def get_message(prompt):
  return [{"role": "system", "content": prompt[0]},{"role": "user", "content": prompt[1]}]

### System Prompts

In [ ]:
system_prompt_01 = """
تو یک دستار هوشمند برای تشخیص احساسات از متن داده شده هستی. \
با توجه به متن داده شده فقط این خروجی ها ممکن ست : خنثی، مثبت، منفی، سایر. \
در صورتی که نتوانستی به درستی احساسات را تحلیل کنی فقط خروجی سایر را برگردان.
"""

In [ ]:
system_prompt_02 = """
You are an AI asistant for sentiment analysis. \
According to given text only these outputs are possible: NEUTRAL, POSITIVE, NEGATIVE, OTHER. \
If you can not detect the sentiment correctly only return OTHER.
"""

### Zero-Shot User Prompts

In [ ]:
zs_user_prompt_01= """
نظر: REVIEW,
احساس:
"""

In [ ]:
zs_user_prompt_02="""
Review: REVIEW,
Sentiment:
"""

### Few-Shot User Prompts

In [ ]:
diff_df = df.loc[~df.index.isin(sub_df.index)]
def create_examples_df(df, n_samples):
    groups = df['sentiment'].unique()
    sampled = []

    for sentiment in groups:
        group_df = df[df['sentiment'] == sentiment]
        sampled_df = group_df.sample(n=n_samples, random_state=RANDOM_STATE)
        sampled.append(sampled_df)

    return pd.concat(sampled).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

In [ ]:
def create_example_prompts():
  examples_df = create_examples_df(diff_df, 1)
  en_examples = ""
  fa_examples = ""
  for i, row in examples_df.iterrows():
    en_examples += f"Example {i+1}:\n"
    en_examples += f"Review: {row['review']}\n"
    en_examples += f"Sentiment: {row['sentiment']}\n"

    fa_examples += f"مثال {i+1}:\n"
    fa_examples += f"نظر: {row['review']}\n"
    fa_examples += f"احساس: {label_en_fa[row['sentiment']]}\n"

  return en_examples, fa_examples

In [ ]:
en_examples, fa_examples = create_example_prompts()
print(en_examples)
print("*" * 50)
print(fa_examples)

In [ ]:
fs_user_promps_01 = f"""
مثالها: \
{fa_examples}

نظر: REVIEW,
احساس:
"""

In [ ]:
fs_user_promps_02 = f"""
Examples: \
{en_examples}

Review: REVIEW,
Sentiment:
"""

### Create Prompts

In [ ]:
zero_shot_combination = [(system_prompt_01, zs_user_prompt_01), (system_prompt_02, zs_user_prompt_02)]
few_shot_combination = [(system_prompt_01, fs_user_promps_01), (system_prompt_02, fs_user_promps_02)]
zero_shot_final_promps = [get_message(prompt) for prompt in zero_shot_combination]
few_shot_final_promps = [get_message(prompt) for prompt in few_shot_combination]